In [1]:
# ===============================================================
# Recycling & Organic Waste Recovery — Yearly Summary Version
# Dataset: Daily Solid Waste (boralesgamuwa_2012-2018.csv)
# Flow: Raw → Cleaning → Standardization → Simulation → Outputs
# ===============================================================

import pandas as pd
from pathlib import Path
import time

# ------------------- CONFIG -------------------
CSV_PATH = "boralasgamuwa_2012-2018.csv"   # full or relative path

CANDIDATE_COLS = {
    "date": ["date", "collection_date", "Date", "DATE", "ticket_date"],
    "type": ["waste_type", "type", "Waste Type", "waste", "category"],
    "weight": ["weight_kg", "weight", "Weight(kg)", "Quantity(kg)", "kg", "amount_kg", "net_weight_kg"]
}

RECOVERY_RATE = {
    "Plastics": 0.55,
    "Metals":   0.78,
    "Glass":    0.65,
    "E-waste":  0.72,
    "Organic":  0.25,
    "MSW":      0.10
}

LABEL_MAP = {
    # plastics
    "plastic": "Plastics", "plastics": "Plastics", "polythene": "Plastics",
    "polythene/plastic": "Plastics", "polyethylene": "Plastics",
    "pet": "Plastics", "hdpe": "Plastics", "ldpe": "Plastics", "pp": "Plastics",

    # metals
    "metal": "Metals", "metals": "Metals", "iron": "Metals", "scrap iron": "Metals",
    "steel": "Metals", "aluminum": "Metals", "aluminium": "Metals", "copper": "Metals",
    "ferrous": "Metals", "non-ferrous": "Metals",

    # glass
    "glass": "Glass", "bottle glass": "Glass", "broken glass": "Glass",

    # e-waste
    "e-waste": "E-waste", "ewaste": "E-waste", "e waste": "E-waste", "weee": "E-waste",

    # organic waste
    "organic": "Organic", "sorted organic waste": "Organic", "food waste": "Organic",
    "compostable": "Organic", "biodegradable": "Organic",

    # msw
    "msw": "MSW", "mixed waste": "MSW", "municipal solid waste": "MSW"
}
# ---------------------------------------------------------------


def pick_column(df: pd.DataFrame, options: list):
    for c in options:
        if c in df.columns:
            return c
    raise KeyError(f"None of these columns were found: {options}\nAvailable: {list(df.columns)}")


def load_csv(path: str) -> pd.DataFrame:
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p.resolve()}")
    return pd.read_csv(p)


def clean(df: pd.DataFrame) -> pd.DataFrame:
    col_date = pick_column(df, CANDIDATE_COLS["date"]) if any(c in df.columns for c in CANDIDATE_COLS["date"]) else None
    col_type = pick_column(df, CANDIDATE_COLS["type"])
    col_wt   = pick_column(df, CANDIDATE_COLS["weight"])

    keep = [c for c in [col_date, col_type, col_wt] if c is not None]
    df = df[keep].copy()

    rename = {}
    if col_date: rename[col_date] = "date"
    rename[col_type] = "type_raw"
    rename[col_wt]   = "collected_kg"
    df = df.rename(columns=rename)

    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["type_raw"] = df["type_raw"].astype(str).str.strip().str.lower()
    df["collected_kg"] = pd.to_numeric(df["collected_kg"], errors="coerce")

    df = df.dropna(subset=["type_raw", "collected_kg"])
    df = df[df["collected_kg"] >= 0]
    return df


def standardize(df: pd.DataFrame) -> pd.DataFrame:
    df["material_std"] = df["type_raw"].map(LABEL_MAP)

    # Regex fallback for edge cases
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("plastic|poly", regex=True), "material_std"] = "Plastics"
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("metal|steel|alum|copper|iron", regex=True), "material_std"] = "Metals"
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("glass", regex=True), "material_std"] = "Glass"
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("e-?waste|weee", regex=True), "material_std"] = "E-waste"
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("organic|food|bio", regex=True), "material_std"] = "Organic"
    df.loc[df["material_std"].isna() & df["type_raw"].str.contains("msw|mixed", regex=True), "material_std"] = "MSW"

    unknowns = df[df["material_std"].isna()]["type_raw"].unique()
    if len(unknowns) > 0:
        print("\n⚠️ Unrecognized waste types (not mapped):")
        for val in unknowns:
            print(" -", val)

    df = df[df["material_std"].notna()].copy()
    df["recovery_rate"] = df["material_std"].map(RECOVERY_RATE)
    df = df.dropna(subset=["recovery_rate"])
    return df


def simulate(df: pd.DataFrame) -> pd.DataFrame:
    df["recovered_kg"] = (df["collected_kg"] * df["recovery_rate"]).round(2)
    return df


def summarize(df: pd.DataFrame):
    # --- Overall Summary by Material ---
    by_mat = df.groupby("material_std", as_index=False).agg(
        collected_kg=("collected_kg", "sum"),
        recovered_kg=("recovered_kg", "sum")
    )
    by_mat["recovery_efficiency_%"] = (by_mat["recovered_kg"] / by_mat["collected_kg"] * 100).round(2)

    # --- Year-wise Summary ---
    by_year = None
    if "date" in df.columns and df["date"].notna().any():
        m = df.dropna(subset=["date"]).copy()
        m["year"] = m["date"].dt.year
        by_year = m.groupby(["year", "material_std"], as_index=False).agg(
            collected_kg=("collected_kg", "sum"),
            recovered_kg=("recovered_kg", "sum")
        )
        by_year["recovery_efficiency_%"] = (by_year["recovered_kg"] / by_year["collected_kg"] * 100).round(2)

    return by_mat, by_year


def interpret(by_mat: pd.DataFrame):
    insights = []
    if by_mat.empty:
        return ["No results. Check label mapping or column names."]

    top_eff = by_mat.sort_values("recovery_efficiency_%", ascending=False).iloc[0]
    insights.append(f"Highest recovery efficiency: {top_eff['material_std']} ({top_eff['recovery_efficiency_%']}%).")

    top_mass = by_mat.sort_values("recovered_kg", ascending=False).iloc[0]
    insights.append(f"Largest recovered mass: {top_mass['material_std']} ({top_mass['recovered_kg']:.2f} kg).")

    if "Organic" in by_mat["material_std"].values:
        org_eff = float(by_mat.loc[by_mat["material_std"] == "Organic", "recovery_efficiency_%"].iloc[0])
        insights.append(f"Organic waste recovery ({org_eff}%) reflects composting potential and collection efficiency.")
    return insights


def main():
    print("1) Loading raw CSV…")
    df = load_csv(CSV_PATH)

    print("2) Cleaning data…")
    df = clean(df)

    print("3) Standardizing values…")
    df = standardize(df)

    print("4) Simulating recovery…")
    df = simulate(df)

    print("5) Summarizing outputs (Year-wise)…")
    by_mat, by_year = summarize(df)

    print("\n=== Total Recovery Summary by Material ===")
    print(by_mat.to_string(index=False))

    if by_year is not None and not by_year.empty:
        print("\n=== Year-wise Recovery Summary (first 12 rows) ===")
        print(by_year.head(12).to_string(index=False))

    print("\n6) Interpretation:")
    for tip in interpret(by_mat):
        print("•", tip)

    # --- Save safely with unique timestamp ---
    Path("outputs").mkdir(exist_ok=True)
    timestamp = time.strftime("%Y%m%d_%H%M%S")

    by_mat.to_csv(f"outputs/recovery_by_material_{timestamp}.csv", index=False)
    if by_year is not None:
        by_year.to_csv(f"outputs/recovery_by_year_{timestamp}.csv", index=False)
    df.to_csv(f"outputs/clean_rows_with_recovered_kg_{timestamp}.csv", index=False)

    print(f"\n✅ Saved with timestamp {timestamp}:")
    print(f" - outputs/recovery_by_material_{timestamp}.csv")
    print(f" - outputs/recovery_by_year_{timestamp}.csv")
    print(f" - outputs/clean_rows_with_recovered_kg_{timestamp}.csv")


if __name__ == "__main__":
    main()


1) Loading raw CSV…
2) Cleaning data…
3) Standardizing values…
4) Simulating recovery…
5) Summarizing outputs (Year-wise)…

=== Total Recovery Summary by Material ===
material_std  collected_kg  recovered_kg  recovery_efficiency_%
         MSW    62870940.0     6287094.0                   10.0
     Organic     3342640.0      835660.0                   25.0

=== Year-wise Recovery Summary (first 12 rows) ===
 year material_std  collected_kg  recovered_kg  recovery_efficiency_%
 2012          MSW     8981740.0      898174.0                   10.0
 2013          MSW     9246230.0      924623.0                   10.0
 2014          MSW     9730880.0      973088.0                   10.0
 2015          MSW    11240260.0     1124026.0                   10.0
 2016          MSW     7045750.0      704575.0                   10.0
 2016      Organic      392550.0       98137.5                   25.0
 2017          MSW     9384440.0      938444.0                   10.0
 2017      Organic      73884